In [1]:
from torchvision.models import video
from torchvision.models.video import swin3d_t
import torch
from torch import nn


In [7]:
class FaceEncoder(video.SwinTransformer3d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, x):
        # x: B C T H W
        print(x.shape)
        x = self.patch_embed(x)  # B _T _H _W _C
        print(x.shape)
        x = self.pos_drop(x)
        x = self.features(x)  # B _T _H _W _C
        print(x.shape)
        x = self.norm(x)
        x = x.permute(0, 4, 1, 2, 3)  # B, _C, _T, _H, _W
        print(x.shape)
        # x = self.avgpool(x)
        # x = torch.flatten(x, 1)
        # x = self.head(x)
        return x


def face_encoder_t():
    return FaceEncoder(
        patch_size=[2, 4, 4],
        embed_dim=96,
        depths=[2, 2, 6, 2],
        num_heads=[3, 6, 12, 24],
        window_size=[8, 7, 7],
        stochastic_depth_prob=0.1,
    )


class TransNet(nn.Module):
    def __init__(self):
        super().__init__()
        # self.face_encoder = face_encoder_t()
        #
        # self.audio_encoder = swin3d_t(
        #     patch_size=[2, 4, 4],
        #     embed_dim=96,
        #     depths=[2, 2, 6, 2],
        #     num_heads=[3, 6, 12, 24],
        #     window_size=[8, 7, 7],
        #     stochastic_depth_prob=0.1,
        # )
        self.swin3d_t = face_encoder_t()
        self.output_conv = nn.Conv3d(96, 3, kernel_size=1)

    def forward(self, x):
        # faces_embedding = self.face_encoder(faces)
        # audios_embedding = self.audio_encoder(audios)
        x = self.swin3d_t(x)
        print(x.shape)
        x = self.output_conv(x)
        return x


model = TransNet()

In [8]:
vids = torch.randn(8, 3, 16, 224, 224)
output = model(vids)

# print(output.shape)


torch.Size([8, 3, 16, 224, 224])
torch.Size([8, 8, 56, 56, 96])


KeyboardInterrupt: 